In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard fbe67e465375231474a2ad80a4389efc77ecff99

Cloning into 'yolov5'...
remote: Enumerating objects: 15656, done.
remote: Counting objects: 100% (263/263), done.
remote: Compressing objects: 100% (161/161), done.
remote: Total 15656 (delta 137), reused 179 (delta 102), pack-reused 15393
Receiving objects: 100% (15656/15656), 14.66 MiB | 29.83 MiB/s, done.
Resolving deltas: 100% (10661/10661), done.
/content/yolov5
HEAD is now at fbe67e4 Fix `OMP_NUM_THREADS=1` for macOS (#8624)


In [ ]:
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.5 MB/s eta 0:00:00
Setup complete. Using torch 2.0.0+cu118 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15101MB, multi_processor_count=40)


In [ ]:
#follow the link below to get your download code from from Roboflow
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="TBSZIcJvZGLN6tqEEfeO")
project = rf.workspace("tfnet-night-vision").project("mul")
dataset = project.version(1).download("yolov5")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.3 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=58ea9b94eb075bcf8812be198ec7b033607fe521352e9ae0eef962e08b25e41a
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.0.9
    Uninstalling pyparsing-3.0.9:
      Successfully uninstalled pyparsing-3.0.9
  Attempting uninstall: idna
    Found existing installation: idna 3.4
    Uninstalling idna-3.4:
    

loading Roboflow workspace...
loading Roboflow project...

Extracting Dataset Version Zip to Mul-1 in yolov5pytorch:: 100%|██████████| 9593/9593 [00:02<00:00, 3398.63it/s]


In [ ]:
%cd /content/yolov5
#after following the link above, recieve python code with these fields filled in
#from roboflow import Roboflow
#rf = Roboflow(api_key="YOUR API KEY HERE")
#project = rf.workspace().project("YOUR PROJECT")
#dataset = project.version("YOUR VERSION").download("yolov5")
# this is the YAML file Roboflow wrote for us that we're loading into this notebook with our data
%cat {dataset.location}/data.yaml

/content/yolov5
names:
- B
- D
- H
- P
nc: 4
roboflow:
  license: CC BY 4.0
  project: mul
  url: https://universe.roboflow.com/night-vision-mmcmp/mul/dataset/1
  version: 1
  workspace: night-vision-mmcmp
test: ../test/images
train: Mul-1/train/images
val: Mul-1/valid/images


In [ ]:
# define number of classes based on YAML
import yaml
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
#this is the model configuration we will use for our tutorial
%cat //content/yolov5/models/yolov5m.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 0.67  # model depth multiple
width_multiple: 0.75  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]]

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate /content/yolov5/models/Tiny_Feature.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors: 3  # AutoAnchor evolves 3 anchors per P output layer
# YOLOv5 backbone

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [768, 3]],  # 0-P1/2
   [-1, 1, Conv, [768, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [768]],
   [-1, 1, Conv, [768, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [768]],
   [-1, 1, Conv, [768, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [768]],
   [-1, 1, Conv, [768, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [768, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [768, False]],  # 10
  ]

head:
  [[-1, 1, Conv, [512, 1, 1]],  # reduce the number of filters in the head to save computation
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [512, False]],  # 17 (P3/medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 14,6], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [512, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:

# train yolov5s on custom data for 100 epochs
# time its performance
%%time
%cd /content/yolov5/
!python train.py --img 320 --batch-size -1 --epochs 500 --label-smoothing 0.01 --hyp data/hyps/hyp.scratch-med.yaml --data {dataset.location}/data.yaml --cfg ./models/Tiny_Feature.yaml --weights '' --project "/content/gdrive/MyDrive/Runs" --name TF-Net_Journal  --cache

/content/yolov5
Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 688, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/usr/local/lib/python3.10/dist-packages/scipy/sparse/__init__.py", line 283, in <module>
    from . import csgraph
  File "/usr/local/lib/python3.10/dist-packages/scipy/sparse/csgraph/__init__.py", line 185, in <module>
    from ._laplacian import laplacian
  File "/usr/local/lib/python3.10/dist-packages/scipy/sparse/csgraph/_laplacian.py", line 7, in <module>
    from scipy.sparse.linalg import LinearOperator
  File "/usr/local/lib/python3.10/dist-packages/scipy/sparse/linalg/__init__.py", line 120, in <module>
    from ._isolve import *
  File "/usr/local/lib/python3.10/dist-packages/scipy/sparse/linalg/_isolve/__init__.py", line 6, in <module>
    from .lgmres import lgmres
  File "/usr/local/lib/python3.10/

In [ ]:
#Later, if it is stopped prematurely and I need to resume, I use
%%time
%cd /content/yolov5/
!python train.py --img 320 --weights '' --project "/content/gdrive/MyDrive/Runs"  --name TF-NET_Journal_simple_conv  --cache  --resume "/content/gdrive/MyDrive/Runs/TF-Net_Journal/weights/last.pt"

/content/yolov5
2023-05-12 13:26:45.073717: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-12 13:26:45.904274: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
train: weights=, cfg=, data=data/coco128.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=300, batch_size=16, imgsz=320, rect=False, resume=/content/gdrive/MyDrive/Runs/TF-Net_Journal/weights/last.pt, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=/content/gdrive/MyDrive/Runs, name=TF-NET_Journal_simple_conv, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=1

In [ ]:
%cd /content/yolov5/
!python detect.py --weights /content/gdrive/MyDrive/Runs/TF-Net_Journal/weights/best.pt  --img 320 --conf 0.4 --source /content/yolov5/Mul-1/valid/images
#--visualize

/content/yolov5
detect: weights=['/content/gdrive/MyDrive/Runs/TF-Net_Journal/weights/best.pt'], source=/content/yolov5/Mul-1/valid/images, data=data/coco128.yaml, imgsz=[320, 320], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 232 layers, 13046801 parameters, 0 gradients, 169.7 GFLOPs
image 1/240 /content/yolov5/Mul-1/valid/images/00000000_0000000000916B52_w320_h256_pMono14_jpg.rf.0f68cc2e3ebc9f937375918bf07c7aeb.jpg: 320x320 1 P, Done. (0.022s)
image 2/240 /content/yolov5/Mul-1/valid/images/00000000_0000000001247F81_w320_h256_pMono14_jpg.rf.74825af883e555637a59d4a6e76536b6.jpg:

In [ ]:
%cd /content/yolov5/
!python detect.py --weights /content/gdrive/MyDrive/Runs/TF-Net_Journal/weights/best.pt --img 320 --conf 0.4 --source /content/yolov5/Mul-1/train/images
#--visualize

/content/yolov5
detect: weights=['/content/gdrive/MyDrive/Runs/TF-Net_Journal/weights/best.pt'], source=/content/yolov5/Mul-1/train/images, data=data/coco128.yaml, imgsz=[320, 320], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 232 layers, 13046801 parameters, 0 gradients, 169.7 GFLOPs
image 1/4552 /content/yolov5/Mul-1/train/images/00000000_00000000005BC581_w320_h256_pMono14_jpg.rf.6ebc4bdc563a210009fddcaa25aed78b.jpg: 320x320 1 P, Done. (0.022s)
image 2/4552 /content/yolov5/Mul-1/train/images/00000000_00000000005BC581_w320_h256_pMono14_jpg.rf.ca3a94f9650e66b9cde4469d7804ba0b.jp

In [ ]:
!python val.py --task study --weights /content/gdrive/MyDrive/Runs/TF-Net_Journal/weights/best.pt --data {dataset.location}/data.yaml --iou 0.5 --save-txt

val: data=/content/yolov5/Mul-1/data.yaml, weights=['/content/gdrive/MyDrive/Runs/TF-Net_Journal/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.5, task=study, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=True, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False

Running study_data_best.txt --imgsz 256...
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 232 layers, 13046801 parameters, 0 gradients, 169.7 GFLOPs
val: Scanning '/content/yolov5/Mul-1/valid/labels.cache' images and labels... 240 found, 0 missing, 0 empty, 0 corrupt: 100% 240/240 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 8/8 [00:04<00:00,  1.75it/s]
                 all        240        269      0.936      0.855      0.949      0.576
                   B       

In [ ]:
!python val.py --task speed --weights /content/gdrive/MyDrive/Runs/TF-Net_Journal/weights/best.pt --data {dataset.location}/data.yaml --iou 0.5 --save-txt

val: data=/content/yolov5/Mul-1/data.yaml, weights=['/content/gdrive/MyDrive/Runs/TF-Net_Journal/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.5, task=speed, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=True, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 232 layers, 13046801 parameters, 0 gradients, 169.7 GFLOPs
val: Scanning '/content/yolov5/Mul-1/valid/labels.cache' images and labels... 240 found, 0 missing, 0 empty, 0 corrupt: 100% 240/240 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 8/8 [00:06<00:00,  1.17it/s]
                 all        240        269      0.942      0.918      0.967      0.632
                   B        240         89      0.981       0.91      0

In [ ]:
!python val.py --img 320 --weights /content/gdrive/MyDrive/Runs/TF-Net_Journal/weights/best.pt --data {dataset.location}/data.yaml --iou 0.5 --verbose --save-txt

val: data=/content/yolov5/Mul-1/data.yaml, weights=['/content/gdrive/MyDrive/Runs/TF-Net_Journal/weights/best.pt'], batch_size=32, imgsz=320, conf_thres=0.001, iou_thres=0.5, task=val, device=, workers=8, single_cls=False, augment=False, verbose=True, save_txt=True, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.10.11 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 232 layers, 13046801 parameters, 0 gradients, 169.7 GFLOPs
val: Scanning '/content/yolov5/Mul-1/valid/labels.cache' images and labels... 240 found, 0 missing, 0 empty, 0 corrupt: 100% 240/240 [00:00<?, ?it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 8/8 [00:08<00:00,  1.09s/it]
                 all        240        269      0.949       0.88      0.971      0.611
                   B        240         89      0.977      0.978      0.99

In [ ]:
##I have created a zip file:
!zip -r /content/Drive_simple_conv.zip /content/gdrive/MyDrive/Runs/TF-Net_Journal
!zip -r /content/TF_Simple_Conv.zip /content/yolov5/runs/
##Than I have downloded that zip file:
from google.colab import files
files.download("/content/Drive_simple_conv.zip")
files.download("/content/TF_Simple_Conv.zip")

Streaming output truncated to the last 5000 lines.
  adding: content/yolov5/runs/detect/exp2/20190925_131530_1_4_IR_166_jpg.rf.6f6f737bae8b74d6a7be6047d00da43a.jpg (deflated 7%)
  adding: content/yolov5/runs/detect/exp2/00000419_000000000124B61A_w320_h256_pMono14_jpg.rf.3300f734f9828c66abc9749c7112388b.jpg (deflated 6%)
  adding: content/yolov5/runs/detect/exp2/20190925_133630_1_3_IR_5_jpg.rf.5a0133b5fb9d22633c181bbf2944e5dd.jpg (deflated 6%)
  adding: content/yolov5/runs/detect/exp2/20190925_133630_1_7_IR_729_jpg.rf.6bbeded305079864b9eef34027fd9735.jpg (deflated 10%)
  adding: content/yolov5/runs/detect/exp2/scene02525_jpg.rf.3833ca62f9207702e2d068adf9179d42.jpg (deflated 4%)
  adding: content/yolov5/runs/detect/exp2/00000064_0000000000848306_w320_h256_pMono14_jpg.rf.2ec8a918f358b8869c3ee3683ba26ed6.jpg (deflated 7%)
  adding: content/yolov5/runs/detect/exp2/2022-10-13-127-_png_jpg.rf.87b919d1dfc79a055a0cdbca93c3ff1f.jpg (deflated 9%)
  adding: content/yolov5/runs/detect/exp2/scene025

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>